---
format:
    html:
        embed-resources: true
---

# Cleaning: Part-1: 

The goal now is to post process the job descriptions you collected in the Crawl

In this section you MUST do it manually using traditional data-science cleaning and parsing skills, i.e. without use of ANY LLM tools or wrappers.

The unit of analysis in this case is "job" 

Your job is to write a cleaning script to Parse the Crawl results and create a data-frame which will be saved to `data/processed-jobs-1.csv`, this data-frame will be used for later EDA. 


You should try to extract as many features as possible, here a possible suggestions. 

These features can provide a comprehensive overview when analyzing job postings for trends, patterns, and insights.

You will almost certainly not be able to extract all of these features, but do your best to make the most detailed data set that you can. You can also ADD more features to this list if they come to mind.

- **Job Title**
- **Company Name**
- **Sector/Industry**
- **Location** (City, State, Country)
- **Job Type** (Full-time, Part-time, Contract, Internship)
- **Salary** (Range or Exact)
- **Experience Level** (Entry-level, Mid-level, Senior-level)
- **Education Requirements** (Degree, Major, Certifications)
- **Skills/Technologies Required** (e.g., Python, SQL, Machine Learning, Cloud)
- **Job Responsibilities/Duties**
- **Required Years of Experience**
- **Benefits** (Health insurance, Retirement plans, Paid time off)
- **Remote Work Options** (Remote, Hybrid, On-site)
- **Application Deadline**
- **Job Posting Date**
- **Job Description Length** (Number of words or characters)
- **Keywords/Frequency of Terms**
- **Certifications Required or Preferred** (e.g., AWS Certified, PMP)
- **Team Size** (If mentioned)
- **Company Size** (Small, Medium, Large)
- **Company Reputation/Ranking** (If available)
- **Job Posting Platform** (Where the job was posted, e.g., LinkedIn, Indeed)
- **Company Values or Culture** (Diversity, Innovation, Sustainability)
- **Visa Sponsorship Availability**
- **Interview Process Information** (If mentioned)
- **Expected Start Date**
- **Job Posting Expiry Date**
- **Gender Diversity Language** (If any)
- **Working Hours/Shift Type** (Day shift, Night shift, Flexible hours)
- **Required Language Skills**
- **Job Location Proximity to Major Cities** (If provided)
- **Travel Requirements** (Percentage or Frequency)
- **Team Collaboration Tools Mentioned** (Slack, Zoom, etc.)
- **Reporting Line** (e.g., Reports to Senior Manager)
- **Job Benefits Related to Learning & Development** (e.g., Courses, Training)
- **Company Stock Options** (If offered)
- **Required Soft Skills** (e.g., Communication, Leadership)
- **Company Perks** (Gym membership, Free meals, Company car)
- **Job Posting ID or Reference Number**


While cleaning try to address the following as best you can

- **Missing Data**: Identify missing or null values in features like salary, company name, or location.
- **Duplicates**: Check for duplicate job listings or job descriptions.
- **Inconsistent Formatting**: Ensure consistency in categorical variables (e.g., job titles, location formats).
- **Data Type Validation**: Ensure each feature has the appropriate data type (e.g., salary as numeric, dates as datetime).
- **Outliers**: Detect outliers in salary, years of experience, or job description length.


In [33]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

with open ('googlejobs_alltitles_2024-11-05_22-34-20.json','r') as file:
    crawled_data = json.load(file)

jobs_results = []
for crawled_unit in crawled_data[0:89]:
    new_unit = crawled_unit['results']['jobs_results']
    jobs_results.extend(new_unit)


In [34]:
def extract_salary(description):
    salary_match = re.search(r'(\$[\d,]+(?:\s*-\s*\$[\d,]+)?)', description)
    return salary_match.group(0) if salary_match else np.nan

def extract_experience_level(description):
    if 'entry' in description.lower():
        return 'Entry-level'
    elif 'senior' in description.lower():
        return 'Senior-level'
    elif 'mid' in description.lower():
        return 'Mid-level'
    else:
        return np.nan

def extract_skills(description):
    skills = ['python', 'r', 'sql', 'machine learning', 'deep learning', 'ai', 'cloud', 'statistics']
    found_skills = [skill for skill in skills if skill.lower() in description.lower()]
    return ', '.join(found_skills) if found_skills else np.nan

def extract_responsibilities(job_highlights):
    responsibilities = []
    for highlight in job_highlights:
        if highlight['title'].lower() == 'responsibilities':
            responsibilities.extend(highlight['items'])
    return ', '.join(responsibilities) if responsibilities else np.nan

def extract_benefits(job_highlights):
    benefits = []
    for highlight in job_highlights:
        if highlight['title'].lower() == 'benefits':
            benefits.extend(highlight['items'])
    return ', '.join(benefits) if benefits else np.nan

def extract_posting_date(posted_at):
    if not posted_at:
        return np.nan
        
    try:
        if isinstance(posted_at, str):
            if 'day' in posted_at.lower():
                days = int(re.search(r'(\d+)', posted_at).group(1))
                return (datetime.now() - pd.Timedelta(days=days)).strftime('%Y-%m-%d')
            elif 'hour' in posted_at.lower():
                hours = int(re.search(r'(\d+)', posted_at).group(1))
                return (datetime.now() - pd.Timedelta(hours=hours)).strftime('%Y-%m-%d')
            elif 'minute' in posted_at.lower():
                minutes = int(re.search(r'(\d+)', posted_at).group(1))
                return (datetime.now() - pd.Timedelta(minutes=minutes)).strftime('%Y-%m-%d')
        return np.nan
    except Exception as e:
        print(f"Error processing date: {posted_at}, Error: {str(e)}")
        return np.nan

In [35]:
def clean_job_data(jobs_results):
    job_list = []

    for job in jobs_results:
        job_unit = {}
        
        detected_extensions = job.get('detected_extensions', {})

        job_unit['Job Title'] = job.get('title', '').strip()
        job_unit['Company Name'] = job.get('company_name', '').strip()
        job_unit['Location'] = job.get('location', '').strip()
        job_unit['Job Type'] = job.get('detected_extensions', {}).get('schedule_type', '').strip()
        job_unit['Salary'] = extract_salary(job.get('description', ''))
        job_unit['Experience Level'] = extract_experience_level(job.get('description', ''))
        job_unit['Skills/Technologies Required'] = extract_skills(job.get('description', ''))
        job_unit['Benefits'] = extract_benefits(job.get('job_highlights', []))
        job_unit['Job Responsibilities/Duties'] = extract_responsibilities(job.get('job_highlights', []))
        job_unit['Job Posting Date'] = extract_posting_date(detected_extensions.get('posted_at', ''))
        job_unit['Job Description Length'] = len(job.get('description', ''))
        job_unit['Job Posting Platform'] = ', '.join([apply_option['title'] for apply_option in job.get('apply_options', [])])
        job_unit['Job Posting ID'] = job.get('job_id', '')

        job_list.append(job_unit)

    df = pd.DataFrame(job_list)

    df['Salary'].replace('', np.nan, inplace=True)
    df['Experience Level'].replace('', np.nan, inplace=True)
    df['Skills/Technologies Required'].replace('', np.nan, inplace=True)
    df['Job Responsibilities/Duties'].replace('', np.nan, inplace=True)
    df['Job Posting Date'].replace('', np.nan, inplace=True)
    
    return df

In [36]:
cleaned_data = clean_job_data(jobs_results)
cleaned_data.to_csv('data/processed-jobs-1.csv', index=False)
print(cleaned_data.head())

                                           Job Title     Company Name  \
0                       Data Scientist, Data Science  Cardinal Health   
1             Usability Researcher 2- Data Scientist    Jobs via Dice   
2  Senior Data Scientist – Financial Crimes and T...             USAA   
3                                Data Scientist, R&D      Eight Sleep   
4                       Data Scientist I B - GBS IND  Bank of America   

                   Location   Job Type               Salary Experience Level  \
0  United States (+1 other)  Full-time   $93,500 - $133,600     Senior-level   
1                  Anywhere  Full-time                  NaN     Senior-level   
2      Colorado Springs, CO  Full-time  $138,230 - $248,810     Senior-level   
3                  Anywhere  Full-time                  NaN              NaN   
4                  Anywhere  Full-time                  NaN              NaN   

                        Skills/Technologies Required  \
0  python, r, machine le

/var/folders/_7/dvn1kl9j1_74vy3jnlj_hw580000gn/T/ipykernel_75563/1711049949.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].replace('', np.nan, inplace=True)
/var/folders/_7/dvn1kl9j1_74vy3jnlj_hw580000gn/T/ipykernel_75563/1711049949.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa